# Beamery SQL assessment

## Global imports

In [2]:
import pandas as pd
import sqlite3

## Create database and tables from `csv`

In [3]:
conn = sqlite3.connect('beamery.db')
cursor = conn.cursor()

In [4]:
users_df = pd.read_csv('users.csv', index_col=0)
users_df.to_sql('users', conn)

follows_df = pd.read_csv('follows.csv', index_col=0)
follows_df['date'] = pd.to_datetime(follows_df['date'])
follows_df.to_sql('follows', conn)

conn.commit()

ValueError: Table 'users' already exists.

In [5]:
def print_rows(rows):
    
    for row in rows:
        print(row)

In [6]:
select_users = '''
    SELECT * FROM USERS
'''

rows = cursor.execute(select_users).fetchall()

print_rows(rows)

(1, 'Harry', 'Potter', 'Gryffindor')
(2, 'Ron', 'Wesley', 'Gryffindor')
(3, 'Hermonie', 'Granger', 'Gryffindor')
(4, 'Ginny', 'Weasley', 'Gryffindor')
(5, 'Draco', 'Malfoy', 'Slytherin')
(6, 'Tom', 'Riddle', 'Slytherin')
(7, 'Luna', 'Lovegood', 'Ravenclaw')
(8, 'Cho', 'Chang', 'Ravenclaw')
(9, 'Cedric', 'Diggory', 'Hufflepuff')


In [7]:
select_follows = '''
    SELECT * FROM FOLLOWS
'''

rows = cursor.execute(select_follows).fetchall()

print_rows(rows)

(1, 2, '1993-09-01 00:00:00')
(2, 1, '1989-01-01 00:00:00')
(3, 1, '1993-07-01 00:00:00')
(2, 3, '1994-10-10 00:00:00')
(3, 2, '1995-03-01 00:00:00')
(4, 2, '1988-08-08 00:00:00')
(4, 1, '1988-08-08 00:00:00')
(1, 4, '1994-04-02 00:00:00')
(1, 5, '2000-01-01 00:00:00')
(5, 1, '2000-01-02 00:00:00')
(5, 6, '1986-01-10 00:00:00')
(7, 1, '1990-02-02 00:00:00')
(1, 7, '1996-10-01 00:00:00')
(1, 8, '1993-09-03 00:00:00')
(8, 1, '1995-09-01 00:00:00')
(8, 9, '1995-09-01 00:00:00')
(9, 8, '1996-01-10 00:00:00')
(7, 8, '1993-09-01 00:00:00')
(3, 9, '1996-05-30 00:00:00')
(4, 9, '1996-05-30 00:00:00')


## Answers
### Users per house

In [8]:
select_users_house = '''
    SELECT COUNT(*), u.house
    FROM USERS u
    GROUP BY u.house
    ORDER BY COUNT(*) DESC
'''

rows = cursor.execute(select_users_house).fetchall()

print_rows(rows)

(4, 'Gryffindor')
(2, 'Slytherin')
(2, 'Ravenclaw')
(1, 'Hufflepuff')


### Links before 1st September 1993

In [9]:
select_links_before = '''
    SELECT *
    FROM FOLLOWS f
    WHERE f.date < '1993-09-01'
    ORDER BY f.date DESC
'''

rows = cursor.execute(select_links_before).fetchall()

print_rows(rows)

(3, 1, '1993-07-01 00:00:00')
(7, 1, '1990-02-02 00:00:00')
(2, 1, '1989-01-01 00:00:00')
(4, 2, '1988-08-08 00:00:00')
(4, 1, '1988-08-08 00:00:00')
(5, 6, '1986-01-10 00:00:00')


### Links before 1st September 1993 with user names

In [13]:
select_links_users = '''
    SELECT u1.first_name, u2.first_name, f.date
    FROM FOLLOWS f
    JOIN USERS u1 ON u1.user_id = f.user_id
    JOIN USERS u2 ON u2.user_id = f.follows
    WHERE f.date < '1993-09-01'
    ORDER BY f.date DESC
'''

rows = cursor.execute(select_links_users).fetchall()

print_rows(rows)

('Hermonie', 'Harry', '1993-07-01 00:00:00')
('Luna', 'Harry', '1990-02-02 00:00:00')
('Ron', 'Harry', '1989-01-01 00:00:00')
('Ginny', 'Ron', '1988-08-08 00:00:00')
('Ginny', 'Harry', '1988-08-08 00:00:00')
('Draco', 'Tom', '1986-01-10 00:00:00')


### Followers by user

In [15]:
select_user_followers = '''
    SELECT u.first_name || ' ' || u.last_name as full_name,
        COUNT(*) as followers
    FROM FOLLOWS f
    JOIN USERS u ON u.user_id = f.follows
    WHERE f.date <= '1999-12-31'
    GROUP BY f.follows
    ORDER BY COUNT(*) DESC
'''

rows = cursor.execute(select_user_followers).fetchall()

print_rows(rows)

('Harry Potter', 5)
('Cedric Diggory', 3)
('Cho Chang', 3)
('Ron Wesley', 3)
('Luna Lovegood', 1)
('Tom Riddle', 1)
('Ginny Weasley', 1)
('Hermonie Granger', 1)


### Follows by user

In [17]:
select_user_follows = '''
    SELECT u.first_name || ' ' || u.last_name as full_name,
        COUNT(*) as followers
    FROM FOLLOWS f
    JOIN USERS u ON u.user_id = f.user_id
    GROUP BY f.user_id
    ORDER BY f.user_id ASC
'''

rows = cursor.execute(select_user_follows).fetchall()

print_rows(rows)

('Harry Potter', 5)
('Ron Wesley', 2)
('Hermonie Granger', 3)
('Ginny Weasley', 3)
('Draco Malfoy', 2)
('Luna Lovegood', 2)
('Cho Chang', 2)
('Cedric Diggory', 1)


### Follows to different house

In [21]:
select_follows_diff_house = '''
    SELECT u1.first_name, u2.first_name, f.date
    FROM FOLLOWS f
    JOIN USERS u1 ON u1.user_id = f.user_id
    JOIN USERS u2 ON u2.user_id = f.follows
    WHERE u1.house != u2.house
'''

rows = cursor.execute(select_follows_diff_house).fetchall()

print_rows(rows)

('Harry', 'Draco', '2000-01-01 00:00:00')
('Draco', 'Harry', '2000-01-02 00:00:00')
('Luna', 'Harry', '1990-02-02 00:00:00')
('Harry', 'Luna', '1996-10-01 00:00:00')
('Harry', 'Cho', '1993-09-03 00:00:00')
('Cho', 'Harry', '1995-09-01 00:00:00')
('Cho', 'Cedric', '1995-09-01 00:00:00')
('Cedric', 'Cho', '1996-01-10 00:00:00')
('Hermonie', 'Cedric', '1996-05-30 00:00:00')
('Ginny', 'Cedric', '1996-05-30 00:00:00')


### Unrequited followings

In [32]:
select_unreq_follows = '''
    SELECT f1.*
    FROM FOLLOWS f1
    LEFT JOIN FOLLOWS f2
    ON f1.user_id = f2.follows
    AND f1.follows = f2.user_id
    WHERE f2.user_id IS NULL 
'''

rows = cursor.execute(select_unreq_follows).fetchall()

print_rows(rows)

(3, 1, '1993-07-01 00:00:00')
(4, 2, '1988-08-08 00:00:00')
(5, 6, '1986-01-10 00:00:00')
(7, 8, '1993-09-01 00:00:00')
(3, 9, '1996-05-30 00:00:00')
(4, 9, '1996-05-30 00:00:00')
